In [57]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [58]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [59]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [60]:
# Use the citipy module to determine city based on latitude and longitude.
#  6.1.4
from citipy import citipy

In [61]:
# Create a list for holding the cities. 6.1.5
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

603

In [62]:
# 6.2.2
import requests
requests.__version__

'2.25.1'

In [63]:
# Import the API key. 6.2.3
from config import weather_api_key

In [64]:
# Starting URL for Weather Map API Call. 6.2.3
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0546ad5dbee13bad86394b7d864b452c


In [65]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [66]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters. We want the first iteration of the logging for each recorded response and the set to start at 1
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [67]:
# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes (while grouping our records in sets of 50).
    if (i % 50 == 0 and i >= 50): #when we get to 50
    #If the statement is true, then the set_count and the record_count are incremented by 1
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    #city_url = url + "&q=" + cities[i]
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}") #removing the blank spaces in the city name 
    # and concatenating the city name. This will find the corresponding weather data for the city 
    #instead of finding the weather data for the first part of the city name
    # Add 1 to the record count.
    record_count += 1

Processing Record 1 of Set 1 | tuktoyaktuk
Processing Record 2 of Set 1 | bethel
Processing Record 3 of Set 1 | mareeba
Processing Record 4 of Set 1 | punta arenas
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | lompoc
Processing Record 7 of Set 1 | carnarvon
Processing Record 8 of Set 1 | qaanaaq
Processing Record 9 of Set 1 | flinders
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | dingle
Processing Record 12 of Set 1 | hithadhoo
Processing Record 13 of Set 1 | aykhal
Processing Record 14 of Set 1 | laguna
Processing Record 15 of Set 1 | cantagallo
Processing Record 16 of Set 1 | mount gambier
Processing Record 17 of Set 1 | nantucket
Processing Record 18 of Set 1 | puerto ayora
Processing Record 19 of Set 1 | almaznyy
Processing Record 20 of Set 1 | beeskow
Processing Record 21 of Set 1 | atuona
Processing Record 22 of Set 1 | godfrey
Processing Record 23 of Set 1 | chifeng
Processing Record 24 of Set 1 | thompson
Processing Record 25 of 

In [68]:
# Run an API request for each of the cities. If there is no weather data for the city, i.e. , 
# a <Response [404]> then there is no weather to retrieve and City not found. Skipping... is printed.

try:
    # Parse the JSON and retrieve data.
    city_weather = requests.get(city_url).json()
    # Parse out the needed data.
    city_lat = city_weather["coord"]["lat"]
    city_lng = city_weather["coord"]["lon"]
    city_max_temp = city_weather["main"]["temp_max"]
    city_humidity = city_weather["main"]["humidity"]
    city_clouds = city_weather["clouds"]["all"]
    city_wind = city_weather["wind"]["speed"]
    city_country = city_weather["sys"]["country"]
    # Convert the date to ISO standard.
    city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
    # Append the city information into city_data list.
    city_data.append({"City": city.title(),
                      "Lat": city_lat,
                      "Lng": city_lng,
                      "Max Temp": city_max_temp,
                      "Humidity": city_humidity,
                      "Cloudiness": city_clouds,
                      "Wind Speed": city_wind,
                      "Country": city_country,
                      "Date": city_date})

# If an error is experienced, skip the city.
except:
    print("City not found. Skipping...")
    pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Data Retrieval Complete      
-----------------------------


In [69]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aflao,6.116,1.1914,82.53,74,40,14.97,TG,2021-07-31 11:35:10


In [70]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# Import linregress
from scipy.stats import linregress

# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):

    # Run regression on hemisphere weather data.
    (slope, intercept, r_value, p_value, std_err) = linregress(x_values, y_values)

    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x_values * slope + intercept
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Create a scatter plot and plot the regression line.
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")
    plt.xlabel('Latitude')
    plt.ylabel(y_label)
    plt.show()